In [1]:
from string import digits
import acquire
import prepare
import requests
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

import re
import unicodedata
import nltk

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## Acquire Function

In [2]:
# def make_soup(url):
#     '''
#     This helper function takes in a url and requests and parses HTML
#     returning a soup object.
#     '''
#     response = get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     return soup

# def get_readme_articles(urls, cached=False):
#     '''
#     This function takes in a list of GitHub Repo urls and a parameter
#     with default cached == False which scrapes the title, text, and language for each url, 
#     creates a list of dictionary of features,converts list to df, and returns df.
#     If cached == True, the function returns a df from a json file.
#     '''
#     if cached == True:
#         df = pd.read_json('project_readme.json')
        
#     # cached == False completes a fresh scrape for df     
#     else:

#         # Create an empty list to hold dictionaries
#         text = []

#         # Loop through each url in our list of urls
#         for url in urls:

#             # Make request and soup object using helper
#             soup = make_soup(url)

#             # Save the title of each repo in variable title
#             title = soup.select('h1', class_="Label Label--outline v-align-middle")[0].text

#             # Save the text in each repo to variable text
#             content = soup.select('article', class_="markdown-body entry-content container-lg")[0].text
            
#             # Save the language of each repo in variable language
#             language = soup.select('li.d-inline:nth-child(1) > a:nth-child(1)')[0].text

#             # Create a dictionary holding the title and content for each blog
#             repo = {'title': title, 'content': content, 'language': language}

#             # Add each dictionary to the articles list of dictionaries
#             text.append(repo)
            
#         # convert our list of dictionaries to a df
#         df = pd.DataFrame(text)

#         # Write df to a json file for faster access
#         df.to_json('project_readme.json')
    
#     return df

## Prepare Function

In [10]:

# Define Functions

def basic_clean(text):
    text = (unicodedata.normalize('NFKD', text.lower())
            .encode('ascii', 'ignore') # ascii to reduce noise
            .decode('utf-8', 'ignore') # decode using utf-8
           )
    return re.sub(r"[^a-z0-9\s]", '', text)

def tokenize(string):
    '''
    This function takes in a string and
    returns a tokenized string.
    '''
    # Create tokenizer.
    tokenizer = nltk.tokenize.ToktokTokenizer()
    
    # Use tokenizer
    string = tokenizer.tokenize(string, return_str=True)
    
    return string

def lemmatize(string):
    '''
    This function takes in string for and
    returns a string with words lemmatized.
    '''
    # Create the lemmatizer.
    wnl = nltk.stem.WordNetLemmatizer()
    
    # Use the lemmatizer on each word in the list of words we created by using split.
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    
    # Join our list of words into a string again and assign to a variable.
    string = ' '.join(lemmas)
    
    return string



def remove_stopwords(string, extra_words=[], exclude_words=[]):
    '''
    This function takes in a string, optional extra_words and exclude_words parameters
    with default empty lists and returns a string.
    '''
    # Create stopword_list.
    stopword_list = stopwords.words('english')
    
    # Remove 'exclude_words' from stopword_list to keep these in my text.
    stopword_list = set(stopword_list) - set(exclude_words)
    # Add in 'extra_words' to stopword_list.
    stopword_list = stopword_list.union(set(extra_words))
    
    # Split words in string.
    words = string.split()
    
    # Create a list of words from my string with stopwords removed and assign to variable.
    filtered_words = [word for word in words if word not in stopword_list]
    
    # Join words in the list back into strings and assign to a variable.
    string_without_stopwords = ' '.join(filtered_words)
    
    return string_without_stopwords

#### Create the URL's

- Focused on URL's that were identified as JavaScript or Python
- Opened up searh URL on GitHub, added URLs that fit discription untill 105 were sampled
- Ended on search page 39

In [4]:
urls = ['https://github.com/freeCodeCamp/freeCodeCamp',
        'https://github.com/Famous/famous',
        'https://github.com/vuejs/vue',
        'https://github.com/kevana/ui-for-docker',
        'https://github.com/facebook/react',
        'https://github.com/ipython/ipython',
        'https://github.com/microsoft/TypeScript-Handbook',
        'https://github.com/airbnb/knowledge-repo',
        'https://github.com/rkern/line_profiler',
        'https://github.com/babel/babel-preset-env',
        'https://github.com/s3tools/s3cmd',
        'https://github.com/zzzeek/sqlalchemy',
        'https://github.com/thunil/TecoGAN',
        'https://github.com/l1ving/youtube-dl',
        'https://github.com/mdo/github-buttons',
        'https://github.com/StephenGrider/ReactNativeReduxCasts',
        'https://github.com/apollographql/apollo',
        'https://github.com/wandb/client',
        'https://github.com/RasaHQ/rasa_core',
        'https://github.com/angular-ui/angular-ui-OLDREPO',
        'https://github.com/urwid/urwid',
        'https://github.com/timqian/star-history',
        'https://github.com/PatrickJS/NG6-starter',
        'https://github.com/knrt10/kubernetes-basicLearning',
        'https://github.com/nature-of-code/noc-book',
        'https://github.com/erikbern/git-of-theseus',
        'https://github.com/dortania/OpenCore-Install-Guide',
        'https://github.com/Kapeli/Dash-User-Contributions',
        'https://github.com/github/docs',
        'https://github.com/StephenGrider/redux-code',
        'https://github.com/fossasia/meilix',
        'https://github.com/StephenGrider/EthereumCasts',
        'https://github.com/newren/git-filter-repo',
        'https://github.com/mateodelnorte/meta',
        'https://github.com/arsaboo/homeassistant-config',
        'https://github.com/IronLanguages/main',
        'https://github.com/StephenGrider/FullstackReactCode',
        'https://github.com/openworm/OpenWorm',
        'https://github.com/apache/nano',
        'https://github.com/jupyterhub/repo2docker',
        'https://github.com/abidrahmank/OpenCV2-Python-Tutorials',
        'https://github.com/Ceruleanacg/Personae',
        'https://github.com/mtdvio/ru-tech-chats',
        'https://github.com/xinntao/EDVR',
        'https://github.com/RubensZimbres/Repo-2017',
        'https://github.com/ipfs-inactive/js-ipfs-http-client',
        'https://github.com/browserpass/browserpass-legacy',
        'https://github.com/boston-dynamics/spot-sdk',
        'https://github.com/sourcerer-io/hall-of-fame',
        'https://github.com/blackorbird/APT_REPORT',
        'https://github.com/creationix/howtonode.org',
        'https://github.com/jennschiffer/make8bitart',
        'https://github.com/wdas/reposado',
        'https://github.com/guyzmo/git-repo',
        'https://github.com/Netflix/repokid',
        'https://github.com/nosarthur/gita',
        'https://github.com/harshjv/github-repo-size',
        'https://github.com/babel/babel-standalone',
        'https://github.com/kevin28520/My-TensorFlow-tutorials',
        'https://github.com/diyhue/diyHue',
        'https://github.com/StephenGrider/rn-casts',
        'https://github.com/headsetapp/headset-electron',
        'https://github.com/StijnMiroslav/top-starred-devs-and-repos-to-follow',
        'https://github.com/techgaun/active-forks',
        'https://github.com/donnemartin/viz',
        'https://github.com/tailwindlabs/tailwindui-vue',
        'https://github.com/GitGuardian/gg-shield',
        'https://github.com/dtschust/redux-bug-reporter',
        'https://github.com/burke-software/django-report-builder',
        'https://github.com/antsmartian/lets-build-express',
        'https://github.com/MicrosoftDocs/visualstudio-docs',
        'https://github.com/earwig/git-repo-updater',
        'https://github.com/OpenSourceTogether/Hacktoberfest-2020',
        'https://github.com/lightaime/deep_gcns_torch',
        'https://github.com/A3M4/YouTube-Report',
        'https://github.com/heroku/heroku-repo',
        'https://github.com/lambdaji/tf_repos',
        'https://github.com/StephenGrider/AdvancedReactNative',
        'https://github.com/lightaime/deep_gcns',
        'https://github.com/StephenGrider/DockerCasts',
        'https://github.com/mappum/gitbanner',
        'https://github.com/declare-lab/conv-emotion',
        'https://github.com/MarkWuNLP/MultiTurnResponseSelection',
        'https://github.com/chriswhong/nyctaxi',
        'https://github.com/18F/analytics-reporter',
        'https://github.com/npmhub/npmhub',
        'https://github.com/kesiev/akihabara',
        'https://github.com/ionic-team/ionic-site',
        'https://github.com/ros/rosdistro',
        'https://github.com/GoogleChromeLabs/tooling.report',
        'https://github.com/rpl/flow-coverage-report',
        'https://github.com/storybook-eol/react-native-storybook',
        'https://github.com/StephenGrider/MongoCasts',
        'https://github.com/kundajelab/deeplift',
        'https://github.com/microsoft/HealthClinic.biz',
        'https://github.com/Redocly/create-openapi-repo',
        'https://github.com/philbooth/complexity-report',
        'https://github.com/bensmithett/webpack-css-example',
        'https://github.com/PhantomInsights/mexican-government-report',
        'https://github.com/googlefonts/Inconsolata',
        'https://github.com/laincloud/lain',
        'https://github.com/auth0/repo-supervisor',
        'https://github.com/weightagnostic/weightagnostic.github.io',
        'https://github.com/jdorn/php-reports',
        'https://github.com/joeldenning/coexisting-vue-microfrontends']


In [5]:
print(len(urls)) 

105


#### Create the df

In [6]:
# Here cached == False, so the function will do a fresh scrape of the urls and write data to a json file.

# urls = ['https://github.com/freeCodeCamp/freeCodeCamp',
#         'https://github.com/996icu/996.ICU',
#         'https://github.com/vuejs/vue',
#         'https://github.com/CSolitaire/natural-language-processing-exercises',
#         'https://github.com/facebook/react']

df = acquire.get_readme_articles(urls=urls, cached=True)
df.head()

,title,content,language
0,\n\n\nfreeCodeCamp\n\n/\n\nfreeCodeCamp\n\n,\n\n\n\n\nfreeCodeCamp.org's open-source codeb...,\n\nJavaScript\n91.3%\n
1,\n\n\nFamous\n\n/\n\nfamous\n\n,#THIS REPO IS DEPRECATED\n##Mixed Mode can be ...,\n\nJavaScript\n86.6%\n
2,\n\n\nvuejs\n\n/\n\nvue\n\n,\n\n\n\n\n\n\n\n\n\n\nSupporting Vue.js\nVue.j...,\n\nJavaScript\n97.7%\n
3,\n\n\nkevana\n\n/\n\nui-for-docker\n\n,UI For Docker\n\nThis repo is deprecated. Deve...,\n\nJavaScript\n61.5%\n
4,\n\n\nfacebook\n\n/\n\nreact\n\n,React · \nReact is a JavaScript library for...,\n\nJavaScript\n95.0%\n


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 0 to 104
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     105 non-null    object
 1   content   105 non-null    object
 2   language  105 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


In [11]:
df['title'] = df.title.apply(basic_clean)
df['title'] = df.title.apply(tokenize)
df['title'] = df.title.apply(lemmatize)
df['language'] = df.language.apply(basic_clean)
df['language'] = df.language.apply(tokenize)
df['language'] = df.language.apply(lemmatize)
#
remove_digits = str.maketrans('', '', digits)
df['language'] = df['language'].str.translate(remove_digits)
#
df['text_cleaned'] = df.content.apply(basic_clean)
df['text_tokenized'] = df.text_cleaned.apply(tokenize)
df['text_lemmatized'] = df.text_tokenized.apply(lemmatize)
df['text_filtered'] = df.text_lemmatized.apply(remove_stopwords)
# Add column with list of words
words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df.text_filtered]
df = pd.concat([df, pd.DataFrame({'words': words})], axis=1)
# Adds colum with lenght of word list
df['doc_length'] = [len(wordlist) for wordlist in df.words]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 0 to 104
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            105 non-null    object
 1   content          105 non-null    object
 2   language         105 non-null    object
 3   text_cleaned     105 non-null    object
 4   text_tokenized   105 non-null    object
 5   text_lemmatized  105 non-null    object
 6   text_filtered    105 non-null    object
 7   words            105 non-null    object
 8   doc_length       105 non-null    int64 
dtypes: int64(1), object(8)
memory usage: 8.2+ KB


In [ ]:
df

#### Clean df

In [ ]:
#df = clean_data(df)
#df.head()

In [ ]:
#df.info()

****
**Everything is good to go except the 'language' call**
****

In [ ]:
# urls = ['https://github.com/freeCodeCamp/freeCodeCamp',
#         'https://github.com/996icu/996.ICU',
#         'https://github.com/vuejs/vue',
#         'https://github.com/EbookFoundation/free-programming-books',
#         'https://github.com/facebook/react']

In [ ]:
# def make_soup(url):
#     '''
#     This helper function takes in a url and requests and parses HTML
#     returning a soup object.
#     '''
#     response = get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     return soup

In [ ]:
# # Soup Object
# soup = make_soup('https://github.com/facebook/react')
# soup1 = make_soup('https://github.com/996icu/996.ICU')
# soup2 = make_soup('https://github.com/vuejs/vue')
# soup3 = make_soup('https://github.com/EbookFoundation/free-programming-books')

In [ ]:
# # Language
# soup.select('article', class_="markdown-body entry-content container-lg")[0].text

In [ ]:
# soup1.select('li.d-inline:nth-child(1) > a:nth-child(1)')[0].text

In [ ]:
# soup2.select('li.d-inline:nth-child(1) > a:nth-child(1)')[0].text

In [ ]:
# soup3.select('li.d-inline:nth-child(1) > a:nth-child(1)')[0].text

In [ ]:
# def no_digit(s):
#     no_digits = []
#     # Iterate through the string, adding non-numbers to the no_digits list
#     for i in s:
#         if not i.isdigit():
#             no_digits.append(i)
#         return no_digits